In [1]:
from keras_tuner import HyperModel
import keras_tuner as kt

import numpy as np
import pandas as pd
#!pip install keras_tuner
#import keras_tuner

import keras
from keras import models, layers, metrics, Sequential
from keras.layers import Dense, BatchNormalization, Dropout
import tensorflow as tf
from keras.regularizers import l1,l2,l1_l2
from sklearn.metrics import mean_squared_error
from tensorflow.keras.layers import Input, Dense, Multiply, Concatenate
from tensorflow.keras.models import Model

import tensorflow as tf
import random

import keras_tuner
from keras_tuner import HyperModel
import keras_tuner as kt

from sklearn.preprocessing import MinMaxScaler

2023-09-27 22:04:25.084160: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Import Dataset
firm_df = pd.read_pickle('firm_df49.pkl')
firm_df.index = pd.to_datetime(firm_df.index).to_period('M')

In [3]:
char_core = ['acc','agr','beta','bm','cash','cashpr','cfp','chatoia','chcsho','chfeps','chinv',
       'chmom','chpmia','chtx','currat','depr','dy','ear','ep','gma','grcapx','grltnoa',
       'ill','indmom','invest','lev','lgr','maxret','mom12m','mom1m','mom36m','mve','nincr',
       'orgcap','pchgm_pchsale','pchsale_pchinvt','pchsale_pchrect','pchsale_pchxsga',
       'retvol','roaq','roavol','roeq','salecash','saleinv','sgr','sp','std_dolvol','std_turn','turn']

info_list = ['fyear','year','jyear','permno','ticker','comnam','exchcd','exchname','siccd',
       'indname','size_class','mve_m','rf','ret','ret_adj','ret_ex','ret_adj_ex',]

macro_col = ['RPI', 'W875RX1', 'DPCERA3M086SBEA', 'CMRMTSPLx', 'RETAILx','INDPRO', 'IPFPNSS', 'IPFINAL', 'IPCONGD', 'IPDCONGD', 'IPNCONGD',
       'IPBUSEQ', 'IPMAT', 'IPDMAT', 'IPNMAT', 'IPMANSICS', 'IPB51222S','IPFUELS', 'CUMFNS', 'HWI', 'HWIURATIO', 'CLF16OV', 'CE16OV',
       'UNRATE', 'UEMPMEAN', 'UEMPLT5', 'UEMP5TO14', 'UEMP15OV','UEMP15T26', 'UEMP27OV', 'CLAIMSx', 'PAYEMS', 'USGOOD',
       'CES1021000001', 'USCONS', 'MANEMP', 'DMANEMP', 'NDMANEMP','SRVPRD', 'USTPU', 'USWTRADE', 'USTRADE', 'USFIRE', 'USGOVT',
       'CES0600000007', 'AWOTMAN', 'AWHMAN', 'HOUST', 'HOUSTNE','HOUSTMW', 'HOUSTS', 'HOUSTW', 'PERMIT', 'PERMITNE', 'PERMITMW',
       'PERMITS', 'PERMITW', 'AMDMNOx', 'ANDENOx', 'AMDMUOx', 'BUSINVx','ISRATIOx', 'M1SL', 'M2SL', 'M2REAL', 'BOGMBASE', 'TOTRESNS',
       'NONBORRES', 'BUSLOANS', 'REALLN', 'NONREVSL', 'CONSPI', 'S&P 500','S&P: indust', 'S&P div yield', 'S&P PE ratio', 'FEDFUNDS',
       'CP3Mx', 'TB3MS', 'TB6MS', 'GS1', 'GS5', 'GS10', 'AAA', 'BAA','COMPAPFFx', 'TB3SMFFM', 'TB6SMFFM', 'T1YFFM', 'T5YFFM', 'T10YFFM',
       'AAAFFM', 'BAAFFM', 'TWEXAFEGSMTHx', 'EXSZUSx', 'EXJPUSx','EXUSUKx', 'EXCAUSx', 'WPSFD49207', 'WPSFD49502', 'WPSID61',
       'WPSID62', 'OILPRICEx', 'PPICMM', 'CPIAUCSL', 'CPIAPPSL','CPITRNSL', 'CPIMEDSL', 'CUSR0000SAC', 'CUSR0000SAD',
       'CUSR0000SAS', 'CPIULFSL', 'CUSR0000SA0L2', 'CUSR0000SA0L5','PCEPI', 'DDURRG3M086SBEA', 'DNDGRG3M086SBEA', 'DSERRG3M086SBEA',
       'CES0600000008', 'CES2000000008', 'CES3000000008', 'UMCSENTx','DTCOLNVHFNM', 'DTCTHFNM', 'INVEST', 'VIXCLSx', 'dp', 'ep_macro', 'b/m',
       'ntis', 'tbl', 'tms', 'dfy', 'svar']

macro_core =  ['dp', 'ep_macro', 'b/m','ntis', 'tbl', 'tms', 'dfy', 'svar']

def datasplit(df, dataset):
    df_train = df[df.index < pd.Period((str(1995)+"-1"),freq='M')]
    df_val = df[(df.index >= pd.Period((str(1995)+"-1"),freq='M')) & (df.index < pd.Period((str(2000)+"-1"),freq='M'))]
    df_test = df[df.index >= pd.Period((str(2000)+"-1"),freq='M')]
    if dataset == "firm":
        X_train = df_train[char_core]
        X_val = df_val[char_core]
        X_test = df_test[char_core]
    elif dataset == 'macro':
        X_train = df_train[macro_col]
        X_val = df_val[macro_col]
        X_test = df_test[macro_col]
    elif dataset == "firm_macro":
        X_train = df_train[char_core+macro_col]
        X_val = df_val[char_core+macro_col]
        X_test = df_test[char_core+macro_col]
    elif dataset == "firm_macro8":
        X_train = df_train[char_core+macro_core]
        X_val = df_val[char_core+macro_core]
        X_test = df_test[char_core+macro_core] 
    y_train = df_train['predicted_return']
    y_val = df_val['predicted_return']
    y_test = df_test['predicted_return']
    return (X_train, y_train, X_val, y_val, X_test, y_test, )

In [4]:
X_train, y_train, X_val, y_val, X_test, y_test = datasplit(firm_df, dataset = "firm")
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(88347, 49) (88347,) (31851, 49) (31851,) (90007, 49) (90007,)


In [24]:
def my_metric_fn(y_true, y_pred):
    num = tf.reduce_mean(tf.square(y_true - y_pred))
    den = tf.reduce_mean(tf.square(y_true))
    return 1 - num / den

class MyHyperModel(keras_tuner.HyperModel):
    def build(self, hp, n_inputs=49,):
        l1_reg = hp.Choice("l1_ratio", [0.,])
        l2_reg = hp.Choice("l2_ratio", [0.001,])
        dropout = hp.Choice("dropout",[0.1, 0.])
        lr = hp.Choice("learning_rate", [0.001,0.0001])
        
        # Define the hidden layers 
        input_layer = Input(shape=(49,))
        # Define hidden layers
        hidden_layer1 = Dense(64, activation ='relu', kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(input_layer)
        dropout1 = Dropout(dropout)(hidden_layer1)
        hidden_layer2 = Dense(32, activation ='relu', kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(hidden_layer1)
        dropout2 = Dropout(dropout)(hidden_layer2)        
        hidden_layer3 = Dense(16, activation ='relu', kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(hidden_layer2)
        dropout3 = Dropout(dropout)(hidden_layer3)
        hidden_layer4 = Dense(8, activation ='relu', kernel_regularizer=l1_l2(l1=l1_reg,l2=l2_reg))(hidden_layer2)
        dropout4 = Dropout(dropout)(hidden_layer4)
  
        # Concatenate the last hidden layer with the input layer 
        concatenated_layer = Concatenate()([hidden_layer4, input_layer])
        # Define output layer 
        output_layer = Dense(1,)(concatenated_layer)
        # Create the model 
        model = Model(input_layer, output_layer)
        opt = tf.keras.optimizers.Adam(learning_rate=lr)
        model.compile(loss='mean_squared_error', optimizer=opt,metrics=[my_metric_fn])
        return model
    
    def fit(self, hp, model, *args, **kwargs):
        return model.fit(
            *args,
            batch_size=hp.Choice("batch_size", [128,256]),
            **kwargs,
        )

In [25]:
max_trials = 200
train_epochs = 200

tuner = keras_tuner.RandomSearch(MyHyperModel(),objective='val_loss',max_trials=max_trials,overwrite=True,)
tuner.search(X_train, y_train, epochs=train_epochs, validation_data=(X_val, y_val), 
             callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)])

Trial 8 Complete [00h 00m 24s]
val_loss: 0.0146321477368474

Best val_loss So Far: 0.014578191563487053
Total elapsed time: 00h 08m 55s
INFO:tensorflow:Oracle triggered exit


In [26]:
# 10 Trials for Prediction
x_all = np.concatenate((X_train, X_val))
y_all = np.concatenate((y_train, y_val))
best_hps = tuner.get_best_hyperparameters(5)
print(best_hps[0].values)
seed_list = [458, 165, 530, 564, 590, 560, 829, 170, 376, 176]
yhat_df = pd.DataFrame()

for random_seed in seed_list:
    random.seed(random_seed)
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    best_model = tuner.get_best_models(3)[0]
    best_model.fit(x=x_all, y=y_all, epochs=20,
              callbacks=[keras.callbacks.EarlyStopping(monitor='loss',mode='min', patience=2)])
    print(best_model.evaluate(X_test,y_test))
    y_hat = best_model.predict(X_test).reshape(-1)
    yhat_df[random_seed] = y_hat
    print()

# Print out Predictive R^2
y_predict = yhat_df.mean(axis=1).values.reshape(-1)
y_real = y_test

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

{'l1_ratio': 0.0, 'l2_ratio': 0.001, 'dropout': 0.1, 'learning_rate': 0.0001, 'batch_size': 128}
Epoch 1/20
3757/3757 [==============================] - 8s 2ms/step - loss: 0.0128 - my_metric_fn: 9.5559e-04
Epoch 2/20
3757/3757 [==============================] - 7s 2ms/step - loss: 0.0128 - my_metric_fn: 0.0017
Epoch 3/20
3757/3757 [==============================] - 7s 2ms/step - loss: 0.0128 - my_metric_fn: 0.0015
Epoch 4/20
3757/3757 [==============================] - 7s 2ms/step - loss: 0.0128 - my_metric_fn: 0.0017
Epoch 5/20
3757/3757 [==============================] - 8s 2ms/step - loss: 0.0128 - my_metric_fn: 0.0019
Epoch 6/20
3757/3757 [==============================] - 8s 2ms/step - loss: 0.0128 - my_metric_fn: 0.0020
Epoch 7/20
3757/3757 [==============================] - 8s 2ms/step - loss: 0.0128 - my_metric_fn: 0.0023
Epoch 8/20
3757/3757 [==============================] - 8s 2ms/step - loss: 0.0128 - my_metric_fn: 0.0020
Epoch 9/20
3757/3757 [=============================

Epoch 1/20
3757/3757 [==============================] - 10s 2ms/step - loss: 0.0128 - my_metric_fn: 4.2894e-04
Epoch 2/20
3757/3757 [==============================] - 7s 2ms/step - loss: 0.0128 - my_metric_fn: 0.0014
Epoch 3/20
3757/3757 [==============================] - 7s 2ms/step - loss: 0.0128 - my_metric_fn: 0.0019
Epoch 4/20
3757/3757 [==============================] - 7s 2ms/step - loss: 0.0128 - my_metric_fn: 0.0018
Epoch 5/20
3757/3757 [==============================] - 7s 2ms/step - loss: 0.0128 - my_metric_fn: 0.0020
Epoch 6/20
2813/2813 [==============================] - 4s 1ms/step - loss: 0.0157 - my_metric_fn: 0.0021
[0.01565137319266796, 0.0020898268558084965]
2813/2813 [==============================] - 6s 2ms/step

Epoch 1/20
3757/3757 [==============================] - 8s 2ms/step - loss: 0.0128 - my_metric_fn: 8.7726e-04
Epoch 2/20
3757/3757 [==============================] - 8s 2ms/step - loss: 0.0128 - my_metric_fn: 0.0018
Epoch 3/20
3757/3757 [==================

Epoch 1/20
3757/3757 [==============================] - 8s 2ms/step - loss: 0.0128 - my_metric_fn: 0.0010
Epoch 2/20
3757/3757 [==============================] - 7s 2ms/step - loss: 0.0128 - my_metric_fn: 0.0011
Epoch 3/20
3757/3757 [==============================] - 7s 2ms/step - loss: 0.0128 - my_metric_fn: 0.0015
Epoch 4/20
3757/3757 [==============================] - 7s 2ms/step - loss: 0.0128 - my_metric_fn: 0.0016
Epoch 5/20
3757/3757 [==============================] - 7s 2ms/step - loss: 0.0128 - my_metric_fn: 0.0018
Epoch 6/20
3757/3757 [==============================] - 7s 2ms/step - loss: 0.0128 - my_metric_fn: 0.0017
Epoch 7/20
3757/3757 [==============================] - 8s 2ms/step - loss: 0.0128 - my_metric_fn: 0.0017
Epoch 8/20
2813/2813 [==============================] - 5s 2ms/step - loss: 0.0157 - my_metric_fn: 0.0056
[0.01568034663796425, 0.005599538795650005]
2813/2813 [==============================] - 4s 1ms/step

Epoch 1/20
3757/3757 [============================

Epoch 1/20
3757/3757 [==============================] - 8s 2ms/step - loss: 0.0128 - my_metric_fn: 8.8696e-04
Epoch 2/20
3757/3757 [==============================] - 8s 2ms/step - loss: 0.0128 - my_metric_fn: 0.0014
Epoch 3/20
3757/3757 [==============================] - 8s 2ms/step - loss: 0.0128 - my_metric_fn: 0.0015
Epoch 4/20
3757/3757 [==============================] - 8s 2ms/step - loss: 0.0128 - my_metric_fn: 0.0015
Epoch 5/20
3757/3757 [==============================] - 8s 2ms/step - loss: 0.0128 - my_metric_fn: 0.0020
Epoch 6/20
3757/3757 [==============================] - 8s 2ms/step - loss: 0.0128 - my_metric_fn: 0.0018
Epoch 7/20
2813/2813 [==============================] - 5s 2ms/step - loss: 0.0157 - my_metric_fn: 0.0025
[0.01572246477007866, 0.0024911570362746716]
2813/2813 [==============================] - 4s 1ms/step

Epoch 1/20
3757/3757 [==============================] - 9s 2ms/step - loss: 0.0128 - my_metric_fn: 8.5635e-04
Epoch 2/20
3757/3757 [===================

In [11]:

a = np.mean(np.square(y_predict -  y_real))
b = np.mean(np.square(y_real))
print(1-a/b)

0.005455629526289751
